In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time, pyautogui, re, json
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
driver = webdriver.Chrome()
driver.get("https://lactalis-smartit.onbmc.com/smartit/app/#/")
time.sleep(5)
pyautogui.hotkey("Esc")

In [4]:
driver.find_element(By.NAME, "AUTHENTICATION.LOGIN").send_keys("joaquim.junior.terceiro@br.lactalis.com")
driver.find_element(By.NAME, "AUTHENTICATION.PASSWORD").send_keys("L@ctali_S2024")
driver.find_element(By.NAME, "validateButton").click()

In [5]:
driver.find_element(By.XPATH, '//*[@id="main"]/div/div[1]/div[3]/div/button').click()
time.sleep(2)
total_chamados = driver.find_element(By.XPATH, '//*[@id="main"]/div/div[1]/div[3]/div[4]/h3/span').text
blocos_total = int(total_chamados)//75
def list_chamados(html):
    soup = BeautifulSoup(html, 'html.parser')
    chamados = soup.findAll('span', class_=re.compile('ng-binding ng-scope'))[7:]
    chamados_geral = []
    for chamado in chamados:
        chamados_geral.append(chamado.getText())
    return chamados_geral

def ResetBlocos(blocos_total):
    for bloco in range(0, blocos_total):
        driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[2]/div[2]/button').click()

In [8]:
chamados_tel = []
for bloco in range(0, blocos_total+1):
    if bloco == 0:
        chamados_tel.append(list_chamados(driver.page_source))
    elif driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[2]/div[2]/button').text == 'Próximo Bloco':
        driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[2]/div[2]/button').click()
        chamados_tel.append(list_chamados(driver.page_source))
    elif driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[2]/div[2]/button').text != 'Próximo Bloco':
        driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[2]/div[2]/button[2]').click()
        chamados_tel.append(list_chamados(driver.page_source))
ResetBlocos(blocos_total)

In [9]:
dados_chamados_geral = []

for n in range(0, len(chamados_tel)):
    for chamado in chamados_tel[n][8::10]:
        dados_dict = {}
        for item in chamado.split('\n- ')[1:-5]:
            try:
                chave, valor = item.split(':', 1)
                dados_dict[chave.strip()] = valor.strip()
            except:
                chave, valor = item.split('  ', 1)
                dados_dict[chave.strip()] = valor.strip()
                
        
        # Converter o dicionário para JSON
        dados_json = json.dumps(dados_dict, indent=4, ensure_ascii=False)
        dados_chamados_geral.append(dados_dict)
        
pd.DataFrame(dados_chamados_geral)

,Nome do solicitante,Solicitado para,Organização (empresa),Site (Filial),Departamento,Diretoria,Solicitação de periférico de TI,Empresa,Filial,Nome Solicitante,Usuário,Matricula,Centro de Custo,E-Mail,Incidente vinculado,Aprovador,Nome pedido,Modelo equipamento,Informe a peça ou componente a ser adquirido
0,Daniele Dziekaniak Pereira,Daniele Dziekaniak Pereira,LACTALIS - BR,BRA-028-ITAJAI,TECNOLOGIA DA INFORMAÇÃO,Supply,HD SSD,Lactalis,BRA-035-PAVUNA,ESTER MARINHO LAIA,Thayna MENDONCA,88793,4202100,thayna.mendenca@br.lactalis.com,INC000001048393,Felipe Dias,BR_113 - Solicitação peças e componentes,NaN,NaN
1,KATIUCI GABRIELE NEIS,KATIUCI GABRIELE NEIS,NaN,BRA-026-HUNGRIA,NaN,Recursos Humanos,HD SSD,Lactalis,BRA-059-TRES DE MAIOII,Katiuci Gabriele Neis,Ana Carolina Simon,562428,4010503,NaN,INC000001040908,Felipe Dias,BR_113 - Solicitação peças e componentes,NaN,NaN
2,BRENO BRITO MOURA,BRENO BRITO MOURA,NaN,BRA-075-GOIANIA,NaN,Comercial,Memoria Ram,Itambé,BRA-075-GOIANIA,Breno Brito Moura,Diule Fagundes Nunes,82072,3900028,breno.moura.terceiro@br.lactalis.com,INC000001048857,Felipe Dias,BR_113 - Solicitação peças e componentes,10AHS1MP00,NaN
3,BRENO BRITO MOURA,BRENO BRITO MOURA,NaN,BRA-075-GOIANIA,NaN,Comercial,HD SSD,Itambé,BRA-075-GOIANIA,Breno Brito Moura,Diule Fagundes Nunes,82072,3900028,breno.moura.terceiro@br.lactalis.com,INC000001048857,Felipe Dias,BR_113 - Solicitação peças e componentes,NaN,NaN
4,LUAN HENRIQUE NAVES FELIPE,LUAN HENRIQUE NAVES FELIPE,LACTALIS - BR,BRA-071-BELO HORIZONTE,TECNOLOGIA DA INFORMAÇÃO,Comercial,HD SSD,Itambé,BRA-078-PARA DE MINAS,VANESSA GEORDANNA GONCALVES,VANESSA GEORDANNA GONCALVES,88225,3900025,NaN,INC000001048721,Felipe Dias,BR_113 - Solicitação peças e componentes,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,JOAQUIM MARCELO DOS SANTOS JUNIOR,JOAQUIM MARCELO DOS SANTOS JUNIOR,LACTALIS - BR,BRA-026-HUNGRIA,TECNOLOGIA DA INFORMAÇÃO,Tecnologia da Informação,Bateria,Lactalis,ITUMBIARA,Leandro Martins Coelho,Leandro Martins Coelho,423084,4701700,leandro.m.coelho@br.lactalis.com,WO0000000239442,Felipe Dias,BR_113 - Solicitação peças e componentes,Dell\tLatitude 5490 | S/N: 68L4VW2,NaN
109,MARLON IURI RIBEIRO BATISTA,MARLON IURI RIBEIRO BATISTA,NaN,BRA-027-IJUÍ,NaN,Captação,HD SSD,Lactalis,BRA-005-BARRA MANSA,Liria Souza,Liria Souza,561182,4005805,liria.souza@br.lactalis.com,INC000000956742,Felipe Dias,BR_113 - Solicitação peças e componentes,NaN,NaN
110,AUGUSTO MULLER VIDAL DE OLIVEIRA,AUGUSTO MULLER VIDAL DE OLIVEIRA,LACTALIS - BR,BRA-026-HUNGRIA,TECNOLOGIA DA INFORMAÇÃO,Industrial,Memoria Ram,Lactalis,BARRA MANSA,Daniela Rodrigues Davet de Freitas,Daniela Rodrigues Davet de Freitas,563087,4005503,daniela.freitas@br.lactalis.com,INC000000913744,Felipe Dias,BR_113 - Solicitação peças e componentes,OptiPlex 3040 DDR3L Desktop,NaN
111,Alisson Muniz da Silva,Alisson Muniz da Silva,NaN,BRA-037-PORTO ALEGRE,NaN,Recursos Humanos,Memoria Ram | DDR3L 1.35V,Lactalis,Pouso Alto,JOAQUIM MARTINS NETO,JOAQUIM MARTINS NETO,041564950,4012503,joaquim.neto@br.lactalis.com,INC000000876623,Felipe Dias,BR_113 - Solicitação peças e componentes,Dell OptiPlex 3040,NaN


In [10]:
grupo_chamado = []
geral_chamado = []
status_chamado = []
tipo_chamado = []
designado_chamado = []
filial_chamado = []
cliente_chamado = []
criacao_chamado = []
n_chamado = []

for tel in chamados_tel:
    grupo_chamado.extend(tel[9::10])
    geral_chamado.extend(tel[8::10])
    status_chamado.extend(tel[7::10])
    tipo_chamado.extend(tel[6::10])
    designado_chamado.extend(tel[5::10])
    filial_chamado.extend(tel[4::10])
    cliente_chamado.extend(tel[3::10])
    criacao_chamado.extend(tel[2::10])
    n_chamado.extend(tel[1::10])
    
df = pd.DataFrame({
    'Número do Chamado': n_chamado,
    'Cliente': cliente_chamado,
    'Filial': filial_chamado,
    'Designado': designado_chamado,
    'Tipo': tipo_chamado,
    'Status': status_chamado,
    'Data de Criacao': criacao_chamado,
})

df

,Número do Chamado,Cliente,Filial,Designado,Tipo,Status,Data de Criacao
0,WO0000000348231,THAYNA VIEIRA DA SILVA MENDONCA,BRA-035-PAVUNA,Daniele Dziekaniak Pereira,849273 - BR_113 - Solicitação peças e componentes,Pendente,30 de jul de 2024 11:09:58
1,WO0000000348229,KATIUCI GABRIELE NEIS,BRA-026-HUNGRIA,,849262 - BR_113 - Solicitação peças e componentes,Designado,30 de jul de 2024 10:57:49
2,WO0000000348195,DIULE FAGUNDES NUNES,BRA-075-GOIANIA,BRENO BRITO MOURA,849055 - BR_113 - Solicitação peças e componentes,Pendente,30 de jul de 2024 09:45:20
3,WO0000000348193,DIULE FAGUNDES NUNES,BRA-075-GOIANIA,BRENO BRITO MOURA,849054 - BR_113 - Solicitação peças e componentes,Pendente,30 de jul de 2024 09:43:23
4,WO0000000348186,LUAN HENRIQUE NAVES FELIPE,BRA-071-BELO HORIZONTE,,849019 - BR_113 - Solicitação peças e componentes,Designado,30 de jul de 2024 09:10:18
...,...,...,...,...,...,...,...
108,WO0000000282808,Leandro Coelho,BRA-029-ITUMBIARA,JOAQUIM MARCELO DOS SANTOS JUNIOR,723123 - BR_113 - Solicitação peças e componentes,Pendente,11 de jan de 2024 13:40:48
109,WO0000000278728,Liria Souza,BRA-005-BARRA MANSA,MATHEUS LAIDMER DA SILVA,715512 - BR_113 - Solicitação peças e componentes,Pendente,28 de dez de 2023 09:14:27
110,WO0000000266298,Daniela Rodrigues Davet de Freitas,BRA-005-BARRA MANSA,AUGUSTO MULLER VIDAL DE OLIVEIRA,692639 - BR_113 - Solicitação peças e componentes,Pendente,16 de nov de 2023 16:18:59
111,WO0000000261810,Alisson Muniz da Silva,BRA-038-POUSO ALTO,Alisson Muniz da Silva,684128 - BR_113 - Solicitação peças e componentes,Pendente,1 de nov de 2023 10:59:07


In [10]:
def dados_chamados(id:int):
    dados_chamado = []
    for tel in chamados_tel:
        dados_chamado.extend(tel[id::10])
    return dados_chamado

df = pd.DataFrame({
    'Número do Chamado': dados_chamados(0),
    'Cliente': dados_chamados(2),
    'Filial': dados_chamados(3),
    'Grupo designado': dados_chamados(8),
    'Tipo': dados_chamados(5),
    'Status': dados_chamados(6),
    'Data de criação': dados_chamados(1),
})

df

ValueError: All arrays must be of the same length

In [11]:
df_full = pd.concat([df, pd.DataFrame(dados_chamados_geral)], axis=1)
df_full

,Número do Chamado,Cliente,Filial,Designado,Tipo,Status,Data de Criacao,Nome do solicitante,Solicitado para,Organização (empresa),...,Nome Solicitante,Usuário,Matricula,Centro de Custo,E-Mail,Incidente vinculado,Aprovador,Nome pedido,Modelo equipamento,Informe a peça ou componente a ser adquirido
0,WO0000000348231,THAYNA VIEIRA DA SILVA MENDONCA,BRA-035-PAVUNA,Daniele Dziekaniak Pereira,849273 - BR_113 - Solicitação peças e componentes,Pendente,30 de jul de 2024 11:09:58,Daniele Dziekaniak Pereira,Daniele Dziekaniak Pereira,LACTALIS - BR,...,ESTER MARINHO LAIA,Thayna MENDONCA,88793,4202100,thayna.mendenca@br.lactalis.com,INC000001048393,Felipe Dias,BR_113 - Solicitação peças e componentes,NaN,NaN
1,WO0000000348229,KATIUCI GABRIELE NEIS,BRA-026-HUNGRIA,,849262 - BR_113 - Solicitação peças e componentes,Designado,30 de jul de 2024 10:57:49,KATIUCI GABRIELE NEIS,KATIUCI GABRIELE NEIS,NaN,...,Katiuci Gabriele Neis,Ana Carolina Simon,562428,4010503,NaN,INC000001040908,Felipe Dias,BR_113 - Solicitação peças e componentes,NaN,NaN
2,WO0000000348195,DIULE FAGUNDES NUNES,BRA-075-GOIANIA,BRENO BRITO MOURA,849055 - BR_113 - Solicitação peças e componentes,Pendente,30 de jul de 2024 09:45:20,BRENO BRITO MOURA,BRENO BRITO MOURA,NaN,...,Breno Brito Moura,Diule Fagundes Nunes,82072,3900028,breno.moura.terceiro@br.lactalis.com,INC000001048857,Felipe Dias,BR_113 - Solicitação peças e componentes,10AHS1MP00,NaN
3,WO0000000348193,DIULE FAGUNDES NUNES,BRA-075-GOIANIA,BRENO BRITO MOURA,849054 - BR_113 - Solicitação peças e componentes,Pendente,30 de jul de 2024 09:43:23,BRENO BRITO MOURA,BRENO BRITO MOURA,NaN,...,Breno Brito Moura,Diule Fagundes Nunes,82072,3900028,breno.moura.terceiro@br.lactalis.com,INC000001048857,Felipe Dias,BR_113 - Solicitação peças e componentes,NaN,NaN
4,WO0000000348186,LUAN HENRIQUE NAVES FELIPE,BRA-071-BELO HORIZONTE,,849019 - BR_113 - Solicitação peças e componentes,Designado,30 de jul de 2024 09:10:18,LUAN HENRIQUE NAVES FELIPE,LUAN HENRIQUE NAVES FELIPE,LACTALIS - BR,...,VANESSA GEORDANNA GONCALVES,VANESSA GEORDANNA GONCALVES,88225,3900025,NaN,INC000001048721,Felipe Dias,BR_113 - Solicitação peças e componentes,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,WO0000000282808,Leandro Coelho,BRA-029-ITUMBIARA,JOAQUIM MARCELO DOS SANTOS JUNIOR,723123 - BR_113 - Solicitação peças e componentes,Pendente,11 de jan de 2024 13:40:48,JOAQUIM MARCELO DOS SANTOS JUNIOR,JOAQUIM MARCELO DOS SANTOS JUNIOR,LACTALIS - BR,...,Leandro Martins Coelho,Leandro Martins Coelho,423084,4701700,leandro.m.coelho@br.lactalis.com,WO0000000239442,Felipe Dias,BR_113 - Solicitação peças e componentes,Dell\tLatitude 5490 | S/N: 68L4VW2,NaN
109,WO0000000278728,Liria Souza,BRA-005-BARRA MANSA,MATHEUS LAIDMER DA SILVA,715512 - BR_113 - Solicitação peças e componentes,Pendente,28 de dez de 2023 09:14:27,MARLON IURI RIBEIRO BATISTA,MARLON IURI RIBEIRO BATISTA,NaN,...,Liria Souza,Liria Souza,561182,4005805,liria.souza@br.lactalis.com,INC000000956742,Felipe Dias,BR_113 - Solicitação peças e componentes,NaN,NaN
110,WO0000000266298,Daniela Rodrigues Davet de Freitas,BRA-005-BARRA MANSA,AUGUSTO MULLER VIDAL DE OLIVEIRA,692639 - BR_113 - Solicitação peças e componentes,Pendente,16 de nov de 2023 16:18:59,AUGUSTO MULLER VIDAL DE OLIVEIRA,AUGUSTO MULLER VIDAL DE OLIVEIRA,LACTALIS - BR,...,Daniela Rodrigues Davet de Freitas,Daniela Rodrigues Davet de Freitas,563087,4005503,daniela.freitas@br.lactalis.com,INC000000913744,Felipe Dias,BR_113 - Solicitação peças e componentes,OptiPlex 3040 DDR3L Desktop,NaN
111,WO0000000261810,Alisson Muniz da Silva,BRA-038-POUSO ALTO,Alisson Muniz da Silva,684128 - BR_113 - Solicitação peças e componentes,Pendente,1 de nov de 2023 10:59:07,Alisson Muniz da Silva,Alisson Muniz da Silva,NaN,...,JOAQUIM MARTINS NETO,JOAQUIM MARTINS NETO,041564950,4012503,joaquim.neto@br.lactalis.com,INC000000876623,Felipe Dias,BR_113 - Solicitação peças e componentes,Dell OptiPlex 3040,NaN


In [12]:
dia = datetime.today().strftime('%d.%m')
df_full.to_excel(f'Chamados_br_113_{dia}.xlsx', index=False)